 Derived from
 
 https://www.axonlab.org/hcph-sops/data-management/edf-to-bids/
 
 https://github.com/TheAxonLab/hcph-sops/blob/mkdocs/code/eyetracking/convert.py

Make sure the python version >=3.7 to support the statement

In [46]:
from __future__ import annotations 
from pathlib import Path
import pandas as pd
import numpy as np
from pyedfread import read_edf
from collections import defaultdict
from itertools import product, groupby
from warnings import warn
import re

In [47]:
# Global variable from 
# https://github.com/TheAxonLab/hcph-sops/blob/mkdocs/code/eyetracking/convert.py

DEFAULT_EYE = "right"
DEFAULT_FREQUENCY = 1000
DEFAULT_MODE = "P-CR"
DEFAULT_SCREEN = (0, 800, 0, 600)

# EyeLink calibration coordinates from
# https://www.sr-research.com/calibration-coordinate-calculator/
EYELINK_CALIBRATION_COORDINATES = [
    (400, 300),
    (400, 51),
    (400, 549),
    (48, 300),
    (752, 300),
    (48, 51),
    (752, 51),
    (48, 549),
    (752, 549),
    (224, 176),
    (576, 176),
    (224, 424),
    (576, 424),
]

EYE_CODE_MAP = defaultdict(lambda: "unknown", {"R": "right", "L": "left", "RL": "both"})
EDF2BIDS_COLUMNS = {
    "g": '',
    "p": "pupil",
    "h": "href",
    "r": "raw",
    "fg": "fast",
    "fh": "fast_href",
    "fr": "fast_raw",
}

BIDS_COLUMNS_ORDER = (
    [f"eye{num}_{c}_coordinate" for num, c in product((1, 2), ("x", "y"))]
    + [f"eye{num}_pupil_size" for num in (1, 2)]
    + [f"eye{num}_pupil_{c}_coordinate" for num, c in product((1, 2), ("x", "y"))]
    + [f"eye{num}_fixation" for num in (1, 2)]
    + [f"eye{num}_saccade" for num in (1, 2)]
    + [f"eye{num}_blink" for num in (1, 2)]
    + [f"eye{num}_href_{c}_coordinate" for num, c in product((1, 2), ("x", "y"))]
    + [f"eye{num}_{c}_velocity" for num, c in product((1, 2), ("x", "y"))]
    + [f"eye{num}_href_{c}_velocity" for num, c in product((1, 2), ("x", "y"))]
    + [f"eye{num}_raw_{c}_velocity" for num, c in product((1, 2), ("x", "y"))]
    + [f"fast_{c}_velocity" for c in ("x", "y")]
    + [f"fast_{kind}_{c}_velocity" for kind, c in product(("href", "raw"), ("x", "y"))]
    + [f"screen_ppdeg_{c}_coordinate" for c in ("x", "y")]
    + ["timestamp"]
)


Read in the edf file

In [48]:
DATA_PATH = Path("D:\\Eye_Dataset\\Sub001\\230928_anatomical_MREYE_study\\ET_EDF")
edf_name = "JB2.EDF"
file_path = str(DATA_PATH / edf_name)
# file_path = "D:\\Eye_Dataset\\Sub001\\230928_anatomical_MREYE_study\\ET_EDF\\Bold_GR4.edf"

print(file_path)
ori_recording, ori_events, ori_messages = read_edf(file_path)

print(ori_messages)

# print(messages)
ori_messages = ori_messages.rename(
    columns={
        # Normalize weird header names generated by pyedfread
        "message": "trialid",
        "trial": "trial",
        # Convert some BIDS columns
        "time": "timestamp",
    }
)



recording = ori_recording
messages = ori_messages
events = ori_events
print(f'after replacing the column name: {messages}')

D:\Eye_Dataset\Sub001\230928_anatomical_MREYE_study\ET_EDF\JB2.EDF
       time  trial                                            message
0   1973607     -1  !CAL \n>>>>>>> CALIBRATION (HV5,P-CR) FOR RIGH...
1   1973608     -1                           !CAL Calibration points:
2   1973608     -1                !CAL -10.5, -49.1         0,      0
3   1973608     -1                !CAL -10.5, -71.3         0,  -2457
4   1973608     -1                !CAL -10.1, -29.2         0,   2457
5   1973608     -1                !CAL -43.8, -49.2     -3474,      0
6   1973608     -1                !CAL  22.8, -51.0      3474,      0
7   1973608     -1  !CAL eye check box: (L,R,T,B)\n\t  -50    29  ...
8   1973608     -1  !CAL href cal range: (L,R,T,B)\n\t-5211  5211 ...
9   1973608     -1  !CAL Cal coeff:(X=a+bx+cy+dxx+eyy,Y=f+gx+goaly...
10  1973608     -1    !CAL Prenormalize: offx, offy = -10.544 -49.144
11  1973608     -1       !CAL Gains: cx:104.313 lx:104.623 rx:104.587
12  1973608     -1     

# 1 Parsing the messages

In [49]:
messages = messages.rename(
    columns={c: c.strip() for c in messages.columns.values}
).drop_duplicates()

In [50]:
# Extract calibration headers
_cal_hdr = ori_messages.trialid.str.startswith("!CAL")
calibration = ori_messages[_cal_hdr]
# messages = messages.drop(messages.index[_cal_hdr])

In [51]:
# Extracting the StartTime and StopTime metadata.
message_first_trigger = 'start'
message_last_trigger = 'end'
metadata = {
    'StopTime': None,
    'StartTime': None
}

# Find Start time
start_rows = messages.trialid.str.contains(
    message_first_trigger, case=False, regex=True
)
stop_rows = messages.trialid.str.contains(
    message_last_trigger, case=False, regex=True
)

# Extract calibration headers
_cal_hdr = messages.trialid.str.startswith("!CAL")
calibration = messages[_cal_hdr]
messages = messages.drop(messages.index[_cal_hdr])

# Pick the LAST of the start messages
metadata["StartTime"] = (
    int(messages[start_rows].trialid_time.values[-1])
    if start_rows.any()
    else None
)

# Pick the FIRST of the stop messages
metadata["StopTime"] = (
    int(messages[stop_rows].trialid_time.values[0])
    if stop_rows.any()
    else None
)

# Drop start and stop messages from messages dataframe
messages = messages.loc[~start_rows & ~stop_rows, :]

print(metadata)

{'StopTime': None, 'StartTime': None}


In [52]:
# Extracting basic metadata.
# !MODE RECORD CR 1000 2 0 R

mode_record = messages.trialid.str.startswith("!MODE RECORD")

meta_record = {
    "freq": DEFAULT_FREQUENCY,
    "mode": DEFAULT_MODE,
    "eye": DEFAULT_EYE,
}

if mode_record.any():
    try:
        meta_record = re.match(
            r"\!MODE RECORD (?P<mode>\w+) (?P<freq>\d+) \d \d (?P<eye>[RL]+)",
            messages[mode_record].trialid.iloc[-1].strip(),
        ).groupdict()

        meta_record["eye"] = EYE_CODE_MAP[meta_record["eye"]]
        meta_record["mode"] = (
            "P-CR" if meta_record["mode"] == "CR" else meta_record["mode"]
        )
    except AttributeError:
        warn(
            "Error extracting !MODE RECORD message, "
            "using default frequency, mode, and eye"
        )
    finally:
        messages = messages.loc[~mode_record]

eye = (
    ("right", "left") if meta_record["eye"] == "both" else (meta_record["eye"],)
)

metadata["SamplingFrequency"] = int(meta_record["freq"])
metadata["EyeTrackingMethod"] = meta_record["mode"]
metadata["RecordedEye"] = meta_record["eye"]

In [53]:
# Extracting screen parameters.
# GAZE_COORDS 0.00 0.00 800.00 600.00

# Extract GAZE_COORDS message signaling start of recording
gaze_msg = messages.trialid.str.startswith("GAZE_COORDS")

metadata["ScreenAOIDefinition"] = [
    "square",
    DEFAULT_SCREEN,
]
if gaze_msg.any():
    try:
        gaze_record = re.match(
            r"GAZE_COORDS (\d+\.\d+) (\d+\.\d+) (\d+\.\d+) (\d+\.\d+)",
            messages[gaze_msg].trialid.iloc[-1].strip(),
        ).groups()
        metadata["ScreenAOIDefinition"][1] = [
            int(round(float(gaze_record[0]))),
            int(round(float(gaze_record[2]))),
            int(round(float(gaze_record[1]))),
            int(round(float(gaze_record[3]))),
        ]
    except AttributeError:
        warn("Error extracting GAZE_COORDS")
    finally:
        messages = messages.loc[~gaze_msg]
        
print(metadata)

{'StopTime': None, 'StartTime': None, 'SamplingFrequency': 1000, 'EyeTrackingMethod': 'P-CR', 'RecordedEye': 'right', 'ScreenAOIDefinition': ['square', [0, 800, 0, 600]]}


In [54]:
# Extracting parameters of the pupil fit model.
# ELCL_PROC ELLIPSE (5)
# ELCL_EFIT_PARAMS 1.01 4.00  0.15 0.05  0.65 0.65  0.00 0.00 0.30
# Extract ELCL_PROC AND ELCL_EFIT_PARAMS to extract pupil fit method
pupilfit_msg = messages.trialid.str.startswith("ELCL_PROC")

if pupilfit_msg.any():
    try:
        pupilfit_method = [
            val
            for val in messages[pupilfit_msg]
            .trialid.iloc[-1]
            .strip()
            .split(" ")[1:]
            if val
        ]
        metadata["PupilFitMethod"] = pupilfit_method[0].lower()
        metadata["PupilFitMethodNumberOfParameters"] = int(
            pupilfit_method[1].strip("(").strip(")")
        )
    except AttributeError:
        warn("Error extracting ELCL_PROC (pupil fitting method)")
    finally:
        messages = messages.loc[~pupilfit_msg]

pupilfit_msg_params = messages.trialid.str.startswith("ELCL_EFIT_PARAMS")
if pupilfit_msg_params.any():
    rows = messages[pupilfit_msg_params]
    row = rows.trialid.values[-1].strip().split(" ")[1:]
    try:
        metadata["PupilFitParameters"] = [
            tuple(float(val) for val in vals)
            for k, vals in groupby(row, key=bool)
            if k
        ]
    except AttributeError:
        warn("Error extracting ELCL_EFIT_PARAMS (pupil fitting parameters)")
    finally:
        messages = messages.loc[~pupilfit_msg_params]
        
print(metadata)

{'StopTime': None, 'StartTime': None, 'SamplingFrequency': 1000, 'EyeTrackingMethod': 'P-CR', 'RecordedEye': 'right', 'ScreenAOIDefinition': ['square', [0, 800, 0, 600]], 'PupilFitMethod': 'ellipse', 'PupilFitMethodNumberOfParameters': 5, 'PupilFitParameters': [(1.01, 4.0), (0.15, 0.05), (0.65, 0.65), (0.0, 0.0, 0.3)]}


In [55]:
# Calibration validation.
# VALIDATE R 4POINT 4 RIGHT at 752,300 OFFSET 0.35 deg. -8.7,-3.8 pix.
# Extract VALIDATE messages for a calibration validation
validation_msg = messages.trialid.str.startswith("VALIDATE")

if validation_msg.any():
    metadata["ValidationPosition"] = []
    metadata["ValidationErrors"] = []

for i_row, validate_row in enumerate(messages[validation_msg].trialid.values):
    prefix, suffix = validate_row.split("OFFSET")
    validation_eye = (
        f"eye{eye.index('right') + 1}"
        if "RIGHT" in prefix
        else f"eye{eye.index('left') + 1}"
    )
    validation_coords = [
        int(val.strip())
        for val in prefix.rsplit("at", 1)[-1].split(",")
        if val.strip()
    ]
    metadata["ValidationPosition"].append(
        [validation_eye, validation_coords]
    )

    validate_values = [
        float(val)
        for val in re.match(
            r"(-?\d+\.\d+) deg\.\s+(-?\d+\.\d+),(-?\d+\.\d+) pix\.",
            suffix.strip(),
        ).groups()
    ]

    metadata["ValidationErrors"].append(
        (validation_eye, validate_values[0], tuple(validate_values[1:]))
    )
messages = messages.loc[~validation_msg]

print(messages)
print(metadata)

    timestamp  trial                                            trialid
27    1999839     -1  NO Reply is disabled for function eyelink_cal_...
28    2007984     -1                               RECCFG CR 1000 2 0 R
29    2007984     -1                                      ELCLCFG TOWER
31    2007984     -1                                THRESHOLDS R 68 179
{'StopTime': None, 'StartTime': None, 'SamplingFrequency': 1000, 'EyeTrackingMethod': 'P-CR', 'RecordedEye': 'right', 'ScreenAOIDefinition': ['square', [0, 800, 0, 600]], 'PupilFitMethod': 'ellipse', 'PupilFitMethodNumberOfParameters': 5, 'PupilFitParameters': [(1.01, 4.0), (0.15, 0.05), (0.65, 0.65), (0.0, 0.0, 0.3)], 'ValidationPosition': [['eye1', [400, 300]], ['eye1', [400, 51]], ['eye1', [400, 549]], ['eye1', [48, 300]], ['eye1', [752, 300]]], 'ValidationErrors': [('eye1', 0.6, (-14.6, -7.0)), ('eye1', 0.34, (-8.6, -3.0)), ('eye1', 0.21, (-3.5, -4.4)), ('eye1', 0.85, (-14.4, -17.8)), ('eye1', 0.33, (5.7, 6.8))]}


In [56]:
# Extracting final bits of metadata.
# Extract THRESHOLDS messages prior recording and process last
thresholds_msg = messages.trialid.str.startswith("THRESHOLDS")
if thresholds_msg.any():
    metadata["PupilThreshold"] = [None] * len(eye)
    metadata["CornealReflectionThreshold"] = [None] * len(eye)
    thresholds_chunks = (
        messages[thresholds_msg].trialid.iloc[-1].strip().split(" ")[1:]
    )
    eye_index = eye.index(EYE_CODE_MAP[thresholds_chunks[0]])
    metadata["PupilThreshold"][eye_index] = int(thresholds_chunks[-2])
    metadata["CornealReflectionThreshold"][eye_index] = int(
        thresholds_chunks[-1]
    )
messages = messages.loc[~thresholds_msg]
print(messages)
print(metadata)

    timestamp  trial                                            trialid
27    1999839     -1  NO Reply is disabled for function eyelink_cal_...
28    2007984     -1                               RECCFG CR 1000 2 0 R
29    2007984     -1                                      ELCLCFG TOWER
{'StopTime': None, 'StartTime': None, 'SamplingFrequency': 1000, 'EyeTrackingMethod': 'P-CR', 'RecordedEye': 'right', 'ScreenAOIDefinition': ['square', [0, 800, 0, 600]], 'PupilFitMethod': 'ellipse', 'PupilFitMethodNumberOfParameters': 5, 'PupilFitParameters': [(1.01, 4.0), (0.15, 0.05), (0.65, 0.65), (0.0, 0.0, 0.3)], 'ValidationPosition': [['eye1', [400, 300]], ['eye1', [400, 51]], ['eye1', [400, 549]], ['eye1', [48, 300]], ['eye1', [752, 300]]], 'ValidationErrors': [('eye1', 0.6, (-14.6, -7.0)), ('eye1', 0.34, (-8.6, -3.0)), ('eye1', 0.21, (-3.5, -4.4)), ('eye1', 0.85, (-14.4, -17.8)), ('eye1', 0.33, (5.7, 6.8))], 'PupilThreshold': [68], 'CornealReflectionThreshold': [179]}


In [57]:
# Flush the remaining messages as a metadata entry.
# Consume the remainder of messages

if not messages.empty:
    metadata["LoggedMessages"] = [
        (int(msg_timestamp), msg.strip())
        for msg_timestamp, msg in messages[["timestamp", "trialid"]].values
    ]
    
print(messages)
print(metadata)

    timestamp  trial                                            trialid
27    1999839     -1  NO Reply is disabled for function eyelink_cal_...
28    2007984     -1                               RECCFG CR 1000 2 0 R
29    2007984     -1                                      ELCLCFG TOWER
{'StopTime': None, 'StartTime': None, 'SamplingFrequency': 1000, 'EyeTrackingMethod': 'P-CR', 'RecordedEye': 'right', 'ScreenAOIDefinition': ['square', [0, 800, 0, 600]], 'PupilFitMethod': 'ellipse', 'PupilFitMethodNumberOfParameters': 5, 'PupilFitParameters': [(1.01, 4.0), (0.15, 0.05), (0.65, 0.65), (0.0, 0.0, 0.3)], 'ValidationPosition': [['eye1', [400, 300]], ['eye1', [400, 51]], ['eye1', [400, 549]], ['eye1', [48, 300]], ['eye1', [752, 300]]], 'ValidationErrors': [('eye1', 0.6, (-14.6, -7.0)), ('eye1', 0.34, (-8.6, -3.0)), ('eye1', 0.21, (-3.5, -4.4)), ('eye1', 0.85, (-14.4, -17.8)), ('eye1', 0.33, (5.7, 6.8))], 'PupilThreshold': [68], 'CornealReflectionThreshold': [179], 'LoggedMessages': [(199983

# 2 Parsing the recording dataframe

In [58]:
print(recording)
recording = ori_recording

              time  px_left  px_right  py_left  py_right  hx_left  hx_right  \
0        2007985.0 -32768.0  -32768.0 -32768.0  -32768.0 -32768.0   -7936.0   
1        2007986.0 -32768.0  -32768.0 -32768.0  -32768.0 -32768.0   -7936.0   
2        2007987.0 -32768.0  -32768.0 -32768.0  -32768.0 -32768.0   -7936.0   
3        2007988.0 -32768.0  -32768.0 -32768.0  -32768.0 -32768.0   -7936.0   
4        2007989.0 -32768.0  -32768.0 -32768.0  -32768.0 -32768.0   -7936.0   
...            ...      ...       ...      ...       ...      ...       ...   
1115099        0.0      0.0       0.0      0.0       0.0      0.0       0.0   
1115100        0.0      0.0       0.0      0.0       0.0      0.0       0.0   
1115101        0.0      0.0       0.0      0.0       0.0      0.0       0.0   
1115102        0.0      0.0       0.0      0.0       0.0      0.0       0.0   
1115103        0.0      0.0       0.0      0.0       0.0      0.0       0.0   

         hy_left  hy_right  pa_left  ...  fgyvel  f

In [59]:
# Curation of the input dataframe
# Normalize timestamps (should be int and strictly positive)
recording = recording.astype({"time": int})
recording = recording[recording["time"] > 0]

recording = recording.rename(
    columns={
#         # Fix buggy header names generated by pyedfread
#         "fhxyvel": "fhxvel",
#         "frxyvel": "frxvel",
        # Normalize weird header names generated by pyedfread
        "rx": "screen_ppdeg_x_coordinate",
        "ry": "screen_ppdeg_y_coordinate",
        # Convert some BIDS columns
        "time": "timestamp",
    }
)

# Split extra columns from the dataframe
extra = recording[["flags", "input", "htype"]]
recording = recording.drop(columns=["flags", "input", "htype"])

# Remove columns that are always very close to zero
recording = recording.loc[:, (recording.abs() > 1e-8).any(axis=0)]
# Remove columns that are always 1e8 or more
recording = recording.loc[:, (recording.abs() < 1e8).any(axis=0)]
# Replace unreasonably high values with NaNs
recording = recording.replace({1e8: np.nan})

print(recording)

         timestamp  px_left  px_right  py_left  py_right  hx_left  hx_right  \
0          2007985 -32768.0  -32768.0 -32768.0  -32768.0 -32768.0   -7936.0   
1          2007986 -32768.0  -32768.0 -32768.0  -32768.0 -32768.0   -7936.0   
2          2007987 -32768.0  -32768.0 -32768.0  -32768.0 -32768.0   -7936.0   
3          2007988 -32768.0  -32768.0 -32768.0  -32768.0 -32768.0   -7936.0   
4          2007989 -32768.0  -32768.0 -32768.0  -32768.0 -32768.0   -7936.0   
...            ...      ...       ...      ...       ...      ...       ...   
1106532    3114517 -32768.0   -1536.0 -32768.0   -6175.0 -32768.0    -182.0   
1106533    3114518 -32768.0   -1535.0 -32768.0   -6170.0 -32768.0    -181.0   
1106534    3114519 -32768.0   -1533.0 -32768.0   -6169.0 -32768.0    -179.0   
1106535    3114520 -32768.0   -1538.0 -32768.0   -6167.0 -32768.0    -184.0   
1106536    3114521 -32768.0   -1545.0 -32768.0   -6157.0 -32768.0    -189.0   

         hy_left  hy_right  pa_left  pa_right  gx_l

In [60]:
# Remove columns that do not apply (e.g., only one eye recorded).
# Drop one eye's columns if not interested in "both"
print(f'The eye we take care of {eye}')
remove_eye = set(("left", "right")) - set(eye)
if remove_eye:
    remove_eye = remove_eye.pop()  # Drop set decoration
    recording = recording.reindex(
        columns=[c for c in recording.columns if remove_eye not in c]
    )
    
columns = recording.columns
print("Columns:")
print(columns)

The eye we take care of ('right',)
Columns:
Index(['timestamp', 'px_right', 'py_right', 'hx_right', 'hy_right', 'pa_right',
       'gx_right', 'gy_right', 'screen_ppdeg_x_coordinate',
       'screen_ppdeg_y_coordinate', 'frxvel'],
      dtype='object')


In [61]:
# Clean-up pupil size and gaze position. 
# These are the parameters we most likely we care for, so special curation is applied:
screen_resolution = [800, 600]

for eyenum, eyename in enumerate(eye):
    # Clean-up implausible values for pupil area (pa)
    recording.loc[
        recording[f"pa_{eyename}"] < 1, f"pa_{eyename}"
    ] = np.nan
    recording = recording.rename(
        columns={f"pa_{eyename}": f"eye{eyenum + 1}_pupil_size"}
    )
    print(f"pa_{eyename} renamed as: eye{eyenum + 1}_pupil_size")
    # Clean-up implausible values for gaze x position
    recording.loc[
        (recording[f"gx_{eyename}"] < 0)
        | (recording[f"gx_{eyename}"] > screen_resolution[0]),
        f"gx_{eyename}",
    ] = np.nan
    # Clean-up implausible values for gaze y position
    recording.loc[
        (recording[f"gy_{eyename}"] <= 0)
        | (recording[f"gy_{eyename}"] > screen_resolution[1]),
        f"gy_{eyename}",
    ] = np.nan
    
print(recording)

pa_right renamed as: eye1_pupil_size
         timestamp  px_right  py_right  hx_right  hy_right  eye1_pupil_size  \
0          2007985  -32768.0  -32768.0   -7936.0   -7936.0              NaN   
1          2007986  -32768.0  -32768.0   -7936.0   -7936.0              NaN   
2          2007987  -32768.0  -32768.0   -7936.0   -7936.0              NaN   
3          2007988  -32768.0  -32768.0   -7936.0   -7936.0              NaN   
4          2007989  -32768.0  -32768.0   -7936.0   -7936.0              NaN   
...            ...       ...       ...       ...       ...              ...   
1106532    3114517   -1536.0   -6175.0    -182.0     -35.0           2185.0   
1106533    3114518   -1535.0   -6170.0    -181.0     -30.0           2181.0   
1106534    3114519   -1533.0   -6169.0    -179.0     -29.0           2180.0   
1106535    3114520   -1538.0   -6167.0    -184.0     -28.0           2183.0   
1106536    3114521   -1545.0   -6157.0    -189.0     -19.0           2192.0   

           gx_

In [62]:
# Munging columns to comply with BIDS. 
# At this point, the dataframe is almost ready for writing out as BIDS.
# Interpolate BIDS column names
columns = list(
    set(recording.columns)
    - set(
        (
            "timestamp",
            "screen_ppdeg_x_coordinate",
            "screen_ppdeg_y_coordinate",
            "eye1_pupil_size",#pa
            "eye2_pupil_size",#pa
        )
    )
)
bids_columns = []
for eyenum, eyename in enumerate(eye):
    for name in columns:
        colprefix = f"eye{eyenum + 1}" if name.endswith(f"_{eyename}") else ""
        _newname = name.split("_")[0]
        _newname = re.sub(r"([xy])$", r"_\1_coordinate", _newname)
        _newname = re.sub(r"([xy])vel$", r"_\1_velocity", _newname)
        _newname = _newname.split("_", 1)
        _newname[0] = EDF2BIDS_COLUMNS[_newname[0]]
        _newname.insert(0, colprefix)
        bids_columns.append("_".join((_n for _n in _newname if _n)))

# Rename columns to be BIDS-compliant
recording = recording.rename(columns=dict(zip(columns, bids_columns)))

# Reorder columns to render nicely (tracking first, pupil size after)
columns = sorted(
    set(recording.columns.values).intersection(BIDS_COLUMNS_ORDER),
    key=lambda entry: BIDS_COLUMNS_ORDER.index(entry),
)
columns += [c for c in recording.columns.values if c not in columns]
recording = recording.reindex(columns=columns)

print(recording)

         eye1_x_coordinate  eye1_y_coordinate  eye1_pupil_size  \
0                      NaN                NaN              NaN   
1                      NaN                NaN              NaN   
2                      NaN                NaN              NaN   
3                      NaN                NaN              NaN   
4                      NaN                NaN              NaN   
...                    ...                ...              ...   
1106532         381.500000         296.299988           2185.0   
1106533         381.600006         296.899994           2181.0   
1106534         381.700012         296.899994           2180.0   
1106535         381.299988         297.100006           2183.0   
1106536         380.799988         298.000000           2192.0   

         eye1_pupil_x_coordinate  eye1_pupil_y_coordinate  \
0                       -32768.0                 -32768.0   
1                       -32768.0                 -32768.0   
2                       

# 3 Parsing the calibration messages

In [63]:
print(calibration)

    timestamp  trial                                            trialid
0     1973607     -1  !CAL \n>>>>>>> CALIBRATION (HV5,P-CR) FOR RIGH...
1     1973608     -1                           !CAL Calibration points:
2     1973608     -1                !CAL -10.5, -49.1         0,      0
3     1973608     -1                !CAL -10.5, -71.3         0,  -2457
4     1973608     -1                !CAL -10.1, -29.2         0,   2457
5     1973608     -1                !CAL -43.8, -49.2     -3474,      0
6     1973608     -1                !CAL  22.8, -51.0      3474,      0
7     1973608     -1  !CAL eye check box: (L,R,T,B)\n\t  -50    29  ...
8     1973608     -1  !CAL href cal range: (L,R,T,B)\n\t-5211  5211 ...
9     1973608     -1  !CAL Cal coeff:(X=a+bx+cy+dxx+eyy,Y=f+gx+goaly...
10    1973608     -1    !CAL Prenormalize: offx, offy = -10.544 -49.144
11    1973608     -1       !CAL Gains: cx:104.313 lx:104.623 rx:104.587
12    1973608     -1       !CAL Gains: cy:145.928 ty:108.256 by:

In [64]:
# Parse calibration metadata
metadata["CalibrationCount"] = 0
if not calibration.empty:
    warn("Calibration of more than one eye is not implemented")
    calibration.trialid = calibration.trialid.str.replace("!CAL", "")
    calibration.trialid = calibration.trialid.str.strip()

    metadata["CalibrationLog"] = list(
        zip(
            calibration.timestamp.values.astype(int),
            calibration.trialid.values,
        )
    )

    calibrations_msg = calibration.trialid.str.startswith(
        "VALIDATION"
    ) & calibration.trialid.str.contains("ERROR")
    metadata["CalibrationCount"] = calibrations_msg.sum()

    calibration_last = calibration.index[calibrations_msg][-1]
    try:
        meta_calib = re.match(
            r"VALIDATION (?P<ctype>[\w\d]+) (?P<eyeid>[RL]+) (?P<eye>RIGHT|LEFT) "
            r"(?P<result>\w+) ERROR (?P<avg>-?\d+\.\d+) avg\. (?P<max>-?\d+\.\d+) max\s+"
            r"OFFSET (?P<offsetdeg>-?\d+\.\d+) deg\. "
            r"(?P<offsetxpix>-?\d+\.\d+),(?P<offsetypix>-?\d+\.\d+) pix\.",
            calibration.loc[calibration_last, "trialid"].strip(),
        ).groupdict()

        metadata["CalibrationType"] = meta_calib["ctype"]
        metadata["AverageCalibrationError"] = [float(meta_calib["avg"])]
        metadata["MaximalCalibrationError"] = [float(meta_calib["max"])]
        metadata["CalibrationResultQuality"] = [meta_calib["result"]]
        metadata["CalibrationResultOffset"] = [
            float(meta_calib["offsetdeg"]),
            (float(meta_calib["offsetxpix"]), float(meta_calib["offsetypix"])),
        ]
        metadata["CalibrationResultOffsetUnits"] = ["deg", "pixels"]
    except AttributeError:
        warn("Calibration data found but unsuccessfully parsed for results")
        
        
print(calibration)

    timestamp  trial                                            trialid
0     1973607     -1  >>>>>>> CALIBRATION (HV5,P-CR) FOR RIGHT: <<<<...
1     1973608     -1                                Calibration points:
2     1973608     -1                     -10.5, -49.1         0,      0
3     1973608     -1                     -10.5, -71.3         0,  -2457
4     1973608     -1                     -10.1, -29.2         0,   2457
5     1973608     -1                     -43.8, -49.2     -3474,      0
6     1973608     -1                      22.8, -51.0      3474,      0
7     1973608     -1  eye check box: (L,R,T,B)\n\t  -50    29   -76 ...
8     1973608     -1  href cal range: (L,R,T,B)\n\t-5211  5211 -3686...
9     1973608     -1  Cal coeff:(X=a+bx+cy+dxx+eyy,Y=f+gx+goaly+ixx+...
10    1973608     -1         Prenormalize: offx, offy = -10.544 -49.144
11    1973608     -1            Gains: cx:104.313 lx:104.623 rx:104.587
12    1973608     -1            Gains: cy:145.928 ty:108.256 by:

C:\Users\evyji\anaconda3\envs\edfenv\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Calibration of more than one eye is not implemented
  after removing the cwd from sys.path.


# 4 Parsing the events dataframe

In [65]:
# print(events)
print(recording)

# Process events: first generate empty columns
recording["eye1_fixation"] = 0
recording["eye1_saccade"] = 0
recording["eye1_blink"] = 0

# Add fixations
for _, fixation_event in events[
    events["type"] == "fixation"
].iterrows():
    recording.loc[
        (recording["timestamp"] >= fixation_event["start"])
        & (recording["timestamp"] <= fixation_event["end"]),
        "eye1_fixation",
    ] = 1

# Add saccades, and blinks, which are a sub-event of saccades
for _, saccade_event in events[
    events["type"] == "saccade"
].iterrows():
    recording.loc[
        (recording["timestamp"] >= saccade_event["start"])
        & (recording["timestamp"] <= saccade_event["end"]),
        "eye1_saccade",
    ] = 1

    if saccade_event["blink"] == 1:
        recording.loc[
            (recording["timestamp"] >= saccade_event["start"])
            & (recording["timestamp"] <= saccade_event["end"]),
            "eye1_blink",
        ] = 1

         eye1_x_coordinate  eye1_y_coordinate  eye1_pupil_size  \
0                      NaN                NaN              NaN   
1                      NaN                NaN              NaN   
2                      NaN                NaN              NaN   
3                      NaN                NaN              NaN   
4                      NaN                NaN              NaN   
...                    ...                ...              ...   
1106532         381.500000         296.299988           2185.0   
1106533         381.600006         296.899994           2181.0   
1106534         381.700012         296.899994           2180.0   
1106535         381.299988         297.100006           2183.0   
1106536         380.799988         298.000000           2192.0   

         eye1_pupil_x_coordinate  eye1_pupil_y_coordinate  \
0                       -32768.0                 -32768.0   
1                       -32768.0                 -32768.0   
2                       

In [66]:
print(recording)

         eye1_x_coordinate  eye1_y_coordinate  eye1_pupil_size  \
0                      NaN                NaN              NaN   
1                      NaN                NaN              NaN   
2                      NaN                NaN              NaN   
3                      NaN                NaN              NaN   
4                      NaN                NaN              NaN   
...                    ...                ...              ...   
1106532         381.500000         296.299988           2185.0   
1106533         381.600006         296.899994           2181.0   
1106534         381.700012         296.899994           2180.0   
1106535         381.299988         297.100006           2183.0   
1106536         380.799988         298.000000           2192.0   

         eye1_pupil_x_coordinate  eye1_pupil_y_coordinate  \
0                       -32768.0                 -32768.0   
1                       -32768.0                 -32768.0   
2                       

# 5 Write the data into BIDS structure

In [74]:
from copy import deepcopy

metadata['Columns'] = recording.columns.tolist()
print(metadata)
save_metadata = deepcopy(metadata)
# metadata.pop('CalibrationLog', None)
# print(metadata)

{'StopTime': None, 'StartTime': None, 'SamplingFrequency': 1000, 'EyeTrackingMethod': 'P-CR', 'RecordedEye': 'right', 'ScreenAOIDefinition': ['square', [0, 800, 0, 600]], 'PupilFitMethod': 'ellipse', 'PupilFitMethodNumberOfParameters': 5, 'PupilFitParameters': [(1.01, 4.0), (0.15, 0.05), (0.65, 0.65), (0.0, 0.0, 0.3)], 'ValidationPosition': [['eye1', [400, 300]], ['eye1', [400, 51]], ['eye1', [400, 549]], ['eye1', [48, 300]], ['eye1', [752, 300]]], 'ValidationErrors': [('eye1', 0.6, (-14.6, -7.0)), ('eye1', 0.34, (-8.6, -3.0)), ('eye1', 0.21, (-3.5, -4.4)), ('eye1', 0.85, (-14.4, -17.8)), ('eye1', 0.33, (5.7, 6.8))], 'PupilThreshold': [68], 'CornealReflectionThreshold': [179], 'LoggedMessages': [(1999839, 'NO Reply is disabled for function eyelink_cal_result'), (2007984, 'RECCFG CR 1000 2 0 R'), (2007984, 'ELCLCFG TOWER')], 'CalibrationCount': 1, 'CalibrationLog': [(1973607, '>>>>>>> CALIBRATION (HV5,P-CR) FOR RIGHT: <<<<<<<<<'), (1973608, 'Calibration points:'), (1973608, '-10.5, -49.

In [75]:
metadata = save_metadata

In [76]:

if isinstance(metadata, dict):
    for k, v in metadata.items():
        if isinstance(v, dict):
            for _k, _v in v.item():
                print(f"_k_v--{_k}: {type(_v)}")
        elif isinstance(v, list) and k=='CalibrationLog':
            for entry in v:
                print(f'entry: {entry}')
                timestamp, info = entry
                print(f'timestamp: {type(timestamp)}')
                print(f'info: {type(info)}')
                
            
        else:
            print(f"-k-v--{k}:{type(v)}--{v}")
  

# -k-v--CalibrationCount:<class 'numpy.int64'>

def convert_to_int(metadata):
    if 'CalibrationCount' in metadata:
        metadata['CalibrationCount'] = int(metadata['CalibrationCount']) if isinstance(metadata['CalibrationCount'], (np.int32, np.int64, int)) else metadata['CalibrationCount']
    if "CalibrationLog" in metadata:
        metadata["CalibrationLog"] = [(int(x[0]),x[1]) if isinstance(x[0], (np.int32, np.int64, int)) else x for x in metadata['CalibrationLog']]
    return metadata

        
convert_metadata = convert_to_int(metadata)
print(convert_metadata)

-k-v--StopTime:<class 'NoneType'>--None
-k-v--StartTime:<class 'NoneType'>--None
-k-v--SamplingFrequency:<class 'int'>--1000
-k-v--EyeTrackingMethod:<class 'str'>--P-CR
-k-v--RecordedEye:<class 'str'>--right
-k-v--ScreenAOIDefinition:<class 'list'>--['square', [0, 800, 0, 600]]
-k-v--PupilFitMethod:<class 'str'>--ellipse
-k-v--PupilFitMethodNumberOfParameters:<class 'int'>--5
-k-v--PupilFitParameters:<class 'list'>--[(1.01, 4.0), (0.15, 0.05), (0.65, 0.65), (0.0, 0.0, 0.3)]
-k-v--ValidationPosition:<class 'list'>--[['eye1', [400, 300]], ['eye1', [400, 51]], ['eye1', [400, 549]], ['eye1', [48, 300]], ['eye1', [752, 300]]]
-k-v--ValidationErrors:<class 'list'>--[('eye1', 0.6, (-14.6, -7.0)), ('eye1', 0.34, (-8.6, -3.0)), ('eye1', 0.21, (-3.5, -4.4)), ('eye1', 0.85, (-14.4, -17.8)), ('eye1', 0.33, (5.7, 6.8))]
-k-v--PupilThreshold:<class 'list'>--[68]
-k-v--CornealReflectionThreshold:<class 'list'>--[179]
-k-v--LoggedMessages:<class 'list'>--[(1999839, 'NO Reply is disabled for function e

In [80]:
# Load the autoreload extension
%load_ext autoreload
# Set autoreload to update the modules every time before executing a new line of code
%autoreload 2

import importlib
from write_bids_yiwei import EyeTrackingRun, write_bids, write_bids_from_df
out_dir = DATA_PATH
edf_extension = 'EDF'
edf_name = "JB2.EDF"
filename = edf_name.split('.')[0]
print(f'bid filename: {filename}')

write_bids_from_df(
    recording, convert_metadata,
    out_dir,
    filename,
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
bid filename: JB2


('D:\\Eye_Dataset\\Sub001\\230928_anatomical_MREYE_study\\ET_EDF\\JB2.tsv.gz',
 'D:\\Eye_Dataset\\Sub001\\230928_anatomical_MREYE_study\\ET_EDF\\JB2.json')

Now the files are generated.
- EDF Path
    - \<filename\>.EDF
    - \<filename\>.tsv.gz

In [79]:
print(recording)

         eye1_x_coordinate  eye1_y_coordinate  eye1_pupil_size  \
0                      NaN                NaN              NaN   
1                      NaN                NaN              NaN   
2                      NaN                NaN              NaN   
3                      NaN                NaN              NaN   
4                      NaN                NaN              NaN   
...                    ...                ...              ...   
1106532         381.500000         296.299988           2185.0   
1106533         381.600006         296.899994           2181.0   
1106534         381.700012         296.899994           2180.0   
1106535         381.299988         297.100006           2183.0   
1106536         380.799988         298.000000           2192.0   

         eye1_pupil_x_coordinate  eye1_pupil_y_coordinate  \
0                       -32768.0                 -32768.0   
1                       -32768.0                 -32768.0   
2                       